In [5]:
import numpy as np
from datetime import datetime
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from SparseNet import *
from plink_dataloader import *
from sklearn.metrics import matthews_corrcoef
import tables
import scikitplot as skplt
from torch.utils.tensorboard import SummaryWriter

In [10]:
geno_path = "/home/kasper/nas/additional_gwas/severe/data/plink_data/pku_inter99"
pheno_path = "/home/kasper/nas/additional_gwas/severe/phenotypes/hos6.pheno.txt"
topology_path = "/home/kasper/Phenotype_prediction/connectivty.tsv"
model = SparseNet(topology_path = topology_path,
                hidden_features=500)

In [12]:
connections = pd.read_csv(topology_path, sep='\t', dtype = int)
snp_features = connections["snp"].max() + 1
gene_features = connections["gene"].max() + 1

connections = np.array([connections["gene"].values,
                        connections["snp"].values])

connections = torch.tensor(connections).long()
gene_layer = sl.SparseLinear(in_features = snp_features,
                             out_features = gene_features,
                             connectivity = connections)

In [41]:
gene_layer.weights = torch.nn.Parameter(gene_layer.weights.abs())

In [43]:
gene_layer.weights

Parameter containing:
tensor([0.0001, 0.0002, 0.0008,  ..., 0.0014, 0.0008, 0.0001],
       requires_grad=True)

In [8]:
train_ds = load_csv(genotypes, phenotypes, 1)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
data, labels = next(iter(train_dl))
x, y = np.where(data.isnan() == True)
mafs = frq_file["MAF"].iloc[y]
counts = np.random.binomial(n=2, p=mafs)
data[x, y] = torch.DoubleTensor(counts)

/data/kasper/miniconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [11]:
model(data.float())

tensor([ 4.7013e-03, -7.4553e-04, -2.0779e-03,  1.8814e-03, -5.2556e-05,
        -7.2324e-03,  1.3090e-03, -1.9238e-03, -7.8221e-03, -1.3290e-03,
        -1.1580e-02,  1.7742e-03,  5.2600e-03, -4.6541e-03,  1.3191e-03,
        -6.7942e-03,  1.6029e-02, -9.7235e-03, -1.1511e-02,  1.2148e-03,
         8.1826e-03, -3.5860e-03, -1.8261e-04,  8.7140e-03, -9.7077e-03,
         1.7585e-04, -1.1300e-02,  8.8505e-03,  2.1538e-03, -1.0631e-02,
         4.2107e-03,  7.9428e-04,  2.7873e-03, -1.5448e-02,  2.9737e-04,
        -5.8864e-03, -4.2002e-03, -7.5796e-03, -7.0061e-03, -9.4450e-03,
         5.3697e-04,  7.3208e-03, -7.4378e-03,  7.5740e-04, -8.6939e-03,
        -1.1240e-02, -5.0418e-03,  3.9227e-03,  6.1272e-03, -1.4456e-02,
         5.5054e-03,  4.7926e-03, -1.1427e-02, -5.6418e-03, -1.6724e-02,
        -8.3111e-03, -7.3658e-03, -5.7603e-03, -1.0461e-02, -4.6861e-03,
         2.8812e-03, -3.2779e-05, -9.4241e-03,  2.7099e-03],
       grad_fn=<ReshapeAliasBackward0>)

In [ ]:
x, y = np.where(np.isnan(data) == True)
mafs = frq_file["MAF"].iloc[y]
counts = np.random.binomial(n=2, p=mafs)

In [ ]:
data[x, y] = np.array(counts, dtype="float32")
data

In [ ]:
model(data)

In [ ]:
def PearsonsCorrelation(output, labels):

    output = output.sigmoid().round()
    x_diff = output - output.mean()
    y_diff = labels - labels.mean()

    covar = (x_diff * y_diff).sum()
    normalization = (x_diff**2 * y_diff**2).sum().sqrt()
    
    if normalization == 0:
        return normalization
    
    else:
        return covar / normalization

In [ ]:
PearsonsCorrelation(output, labels.float())

In [ ]:
w = torch.tensor(4.284273)
bce_loss = nn.BCEWithLogitsLoss(pos_weight=w)
loss = bce_loss(output, labels.float())

In [ ]:
loss